In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# --- Example Usage with the Beta Distribution ---
from scipy.stats import beta


from alignsim.simulate import (
    generate_uniform_confusion_matrix, 
    generate_annotations, 
    generate_fixed_accuracy_annotations,
    generate_latent_bernoulli_results
)
from alignsim.calculate import (
    calculate_3d_agreement,
    calculate_empirical_accuracy,
    calculate_geirhos_error_matrix,
    calculate_geirhos_metrics,
    calculate_geirhos_from_validations
)

from alignsim.distributions import (
    get_beta_dist
)

from alignsim.experiments_ec import (
    run_accuracy_vs_kappa_sim,
    run_facility_experiment
)

from alignsim.visualise import (
    plot_latent_facility_ec_experiment
)


## Usage

The following cells show some characteristic usage of the functionality

In [13]:
# 1. Setup Parameters
K = 16              # Number of classes
samples_per_class = 80
accuracy = 0.9

# 2. Create Balanced Ground Truth
# Generates [0,0... (80 times), 1,1... (80 times), ... 15,15... (80 times)]
gt = np.repeat(np.arange(K), samples_per_class)

In [14]:
# 3. Define the Confusion Matrix (Uniform Error Distribution)
# Every class has 90% accuracy; 10% error is spread over the other 15 classes
cm_uniform = generate_uniform_confusion_matrix(K, accuracy)

# 4. Generate Synthetic Annotations for Two Independent Observers
# Even though they share the same CM, the stochastic nature of sampling
# makes their specific errors independent.
anno_1 = generate_annotations(gt, cm_uniform)
anno_2 = generate_annotations(gt, cm_uniform)

# 5. Calculate 3D Agreement Tensor and Empirical Matrices
# T: (16, 16, 16) tensor tracking (GT, Obs1, Obs2)
# A: (16, 16) Empirical agreement between Obs1 and Obs2
# C1, C2: Empirical confusion matrices for each observer
T, A, C1, C2 = calculate_3d_agreement(gt, anno_1, anno_2, K=K)

# Quick Verification
print(f"Ground Truth shape: {gt.shape}")
print(f"3D Tensor shape:    {T.shape}")
print(f"Observed Agreement (Trace of A / Total): {np.trace(A) / len(gt):.4f}")

Ground Truth shape: (1280,)
3D Tensor shape:    (16, 16, 16)
Observed Agreement (Trace of A / Total): 0.8039


In [15]:
# Assuming 1280 total samples (16 classes * 80)
total_n = len(gt)
exact_correct = int(0.9 * total_n) # 1152 correct samples

# Generate two observers with EXACTLY 90% accuracy
anno_1 = generate_fixed_accuracy_annotations(gt, exact_correct)
anno_2 = generate_fixed_accuracy_annotations(gt, exact_correct)

# Verify accuracy
T, A, C1, C2 = calculate_3d_agreement(gt, anno_1, anno_2, K=K)
print(f"Observed Agreement (Trace of A / Total): {np.trace(A) / len(gt):.4f}")

print(f"Accuracy 1: {calculate_empirical_accuracy(C1)}")
print(f"Accuracy 2: {calculate_empirical_accuracy(C2)}")

Observed Agreement (Trace of A / Total): 0.8102
Accuracy 1: 0.9
Accuracy 2: 0.9


In [16]:
acc1 = calculate_empirical_accuracy(C1)
acc2 = calculate_empirical_accuracy(C2)

print(f"Empirical Accuracy (System 1): {acc1:.4f}")
print(f"Empirical Accuracy (System 2): {acc2:.4f}")

Empirical Accuracy (System 1): 0.9000
Empirical Accuracy (System 2): 0.9000


In [17]:
# Calculate the Geirhos error matrix from your tensor T
geirhos_mtx = calculate_geirhos_error_matrix(T)

print("Geirhos Error Agreement Matrix:")
print(geirhos_mtx)

# Quick sanity check: does it sum to the total number of samples?
print(f"\nTotal samples: {np.sum(geirhos_mtx)} (Expected: {len(gt)})")

Geirhos Error Agreement Matrix:
[[  12  116]
 [ 116 1036]]

Total samples: 1280 (Expected: 1280)


In [ ]:
metrics = calculate_geirhos_metrics(geirhos_mtx)

print(f"Expected Overlap (c_exp): {metrics['c_exp']:.4f}")
print(f"Observed Overlap (c_obs): {metrics['c_obs']:.4f}")
print(f"Error Consistency:            {metrics['kappa']:.4f}")